reference : reference : https://github.com/NLP-kr/tensorflow-ml-nlp-tf2

In [1]:
import os
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [3]:
DATA_IN_PATH = 'gdrive/My Drive/RogerHeederer/NLP_KR/4.TextClassfication_heederer/data_in/'
DATA_OUT_PATH = 'gdrive/My Drive/RogerHeederer/NLP_KR/4.TextClassfication_heederer/data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [4]:
#전처리된 텍스트를 활용하여 벡터화 한다. 전처리한 넘파이 배열은 워드투백에서 사용하지 않는다.
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [5]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [6]:
sentences = []
for review in reviews: #전체 리뷰를 띄어쓰기 단위로 잘라서 단어 리스트로 만든다
  sentences.append(review.split())

In [7]:
sentences[1]

['classic',
 'war',
 'worlds',
 'timothy',
 'hines',
 'entertaining',
 'film',
 'obviously',
 'goes',
 'great',
 'effort',
 'lengths',
 'faithfully',
 'recreate',
 'h',
 'g',
 'wells',
 'classic',
 'book',
 'mr',
 'hines',
 'succeeds',
 'watched',
 'film',
 'appreciated',
 'fact',
 'standard',
 'predictable',
 'hollywood',
 'fare',
 'comes',
 'every',
 'year',
 'e',
 'g',
 'spielberg',
 'version',
 'tom',
 'cruise',
 'slightest',
 'resemblance',
 'book',
 'obviously',
 'everyone',
 'looks',
 'different',
 'things',
 'movie',
 'envision',
 'amateur',
 'critics',
 'look',
 'criticize',
 'everything',
 'others',
 'rate',
 'movie',
 'important',
 'bases',
 'like',
 'entertained',
 'people',
 'never',
 'agree',
 'critics',
 'enjoyed',
 'effort',
 'mr',
 'hines',
 'put',
 'faithful',
 'h',
 'g',
 'wells',
 'classic',
 'novel',
 'found',
 'entertaining',
 'made',
 'easy',
 'overlook',
 'critics',
 'perceive',
 'shortcomings']

#word2Vec 벡터화

In [8]:
#학습 시 필요한 하이퍼파라미터 설정
num_features = 300    # 각 단어에 대해 임베딩된 벡터의 차원
min_word_count = 40   # 적은 빈도수의 단어들은 학습하지 않는다. 40이상 빈도를 가지는 단어만 학습
num_workers = 4       # 모델 학습 시 학습을 위한 프로세스 개수를 지정한다.
context = 10          # 워드투백을 수행하기 위한 컨텍스트 윈도 크기 지정
downsampling = 1e-3   # 학습을 수행할 때 빠른 학습을 위해 정답 단어 라벨에 대한 다운샘플링 비율 지정. 보통 0.001로 성능이 좋다

In [9]:
#위 파라미터를 이용해서 워드투백 학습시작
!pip install gensim

In [10]:
# 워드투벡 학습 과정 진행 상황을 확인해보기 위해 다음과 같은 로깅 기능 사용
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#로그 수준을 INFO에 맞추어 워드투벡 학습과정에서 로그 메시지를 양식에 맞게 INFO 수준으로 보여주게 된다

In [11]:
from gensim.models import word2vec
print("Training model...") #각각의 단어 리스트들에 대한 벡터값을 가지고 있는 모델이 된다.
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features,
                          min_count = min_word_count, window=context, sample=downsampling)

2020-10-13 16:29:47,123 : INFO : 'pattern' package not found; tag filters are not available for English
2020-10-13 16:29:47,134 : INFO : collecting all words and their counts
2020-10-13 16:29:47,135 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-10-13 16:29:47,435 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2020-10-13 16:29:47,722 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2020-10-13 16:29:47,874 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2020-10-13 16:29:47,875 : INFO : Loading a fresh vocabulary
2020-10-13 16:29:47,921 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2020-10-13 16:29:47,922 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2020-10-13 16:29:47,964 : INFO : deleting the raw counts dictionary of 74065 items
2020-10-13 16:29:47,967 : INFO : sample=0.001 downsamples 30 most-common words
2020-10-13 16:29:47,969 : INFO : downsampling leaves estimated 2494384 word corpus (94.9% of prior 2627273)
2020-10-13 16:29:47,995 : INFO : estimated required memory for 8160 words and 300 dimen

In [ ]:
#모델의 하이퍼파라미터를 설정한 내용을 모델 이름에 담아서 나중에 참고해보자
#Word2Vec.load()를 통해서 모델을 다시 불러와 사용할 수 있다.
model_name = "gdrive/My Drive/RogerHeederer/NLP_KR/4.TextClassfication_heederer/data_out/300features_40minwords_10context"
model.save(model_name)

2020-10-13 12:57:11,494 : INFO : saving Word2Vec object under gdrive/My Drive/RogerHeederer/NLP_KR/4.TextClassfication_heederer/data_out/300features_40minwords_10context, separately None
2020-10-13 12:57:11,496 : INFO : not storing attribute vectors_norm
2020-10-13 12:57:11,500 : INFO : not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-10-13 12:57:11,896 : INFO : saved gdrive/My Drive/RogerHeederer/NLP_KR/4.TextClassfication_heederer/data_out/300features_40minwords_10context


이제 만들어진 word2vec 모델을 활용해서 선형 회귀 모델을 학습해보겠다. 우선 학습을 위해서는 리뷰들을 같은 형태의 입력값으로 만들어야 한다. 리뷰마다 단어 개수가 모두 다르기 때문에, 입력값을 하나의 형태로 만드는 방법이 필요

문장에 있는 모든 단어의 벡터값에 대해 평균을 내서 리뷰 하나당 하나의 벡터로 만드는 함수

In [12]:
#words : 단어 모음인 하나의 리뷰가 들어감
#model : 임베딩된 word2vec 모델을 넣는 곳
#num_features : word2vec으로 임베딩할 때 정했던 벡터의 차원수
def get_features(words, model, num_features):
  feature_vector = np.zeros((num_features), dtype=np.float32) #출력 벡터 초기화
  num_words=0
  index2word_set = set(model.wv.index2word) #어휘 사전 준비 / 문장의 단어가 해당 모델 단어 사전에 속하는지 확인하기 위해
                                          
  for w in words: 
    if w in index2word_set: #리뷰의 단어 하나 하나를 word2vec에 있는 단어인지 체크 한다.
      num_words += 1 #있다면 단어 숫자를 1 올려주고
      feature_vector = np.add(feature_vector, model[w]) # word2vec에서 그 단어의 벡터값을 가져와 더해준다.

  feature_vector = np.divide(feature_vector, num_words) #단어 벡터값들의 합을, 단어 개수만큼 나눠준다(평균)
  return feature_vector

In [13]:
# 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수 정의
# reviews: 학습 데이터인 전체 리뷰 데이터를 입력받는 인자

def get_dataset(reviews, model, num_features):
  dataset = list()

  for s in reviews:
    dataset.append(get_features(s, model, num_features))
  
  reviewFeatureVecs = np.stack(dataset) #각 리뷰를 라인 바이 라인으로 벡터를 구해 쌓아놓는다.
  return reviewFeatureVecs

In [14]:
#이제 위 두함수를 사용해서 실제 학습에 사용될 입력값을 만들어보자
test_data_vecs = get_dataset(sentences, model, num_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [15]:
test_data_vecs

array([[ 0.23184285, -0.1198298 , -0.16366903, ...,  0.08386903,
        -0.10069377, -0.09224693],
       [-0.06559019, -0.23833539, -0.30501917, ..., -0.28208685,
        -0.07988952,  0.02504665],
       [-0.08757322,  0.04175428,  0.06635428, ...,  0.14136074,
        -0.04709114, -0.11670062],
       ...,
       [ 0.10825901, -0.11592989, -0.3537106 , ...,  0.13039844,
        -0.06310982, -0.07527832],
       [ 0.0627091 , -0.2710136 , -0.02810978, ...,  0.11511368,
        -0.06486975, -0.17145921],
       [ 0.00709259, -0.30465266, -0.26342863, ...,  0.0161927 ,
         0.08599084,  0.02401569]], dtype=float32)

이제 만들어진 데이터를 가지고 학습 데이터, 검증 데이터를 나눠보자

In [16]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [20]:
X_train.shape, X_test.shape

((20000, 300), (5000, 300))

In [21]:
y_train.shape, y_test.shape

((20000,), (5000,))

In [24]:
#이제 word2vec을 적용한 리뷰들을 모델에 투입하여 학습시켜보자
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
predicted = lgs.predict(X_test)
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

------------
Accuracy: 0.864200
Precision: 0.857420
Recall: 0.876141
F1-Score: 0.866680
AUC: 0.934073


이제 캐글에 제출하기 위한 데이터 만들기

In [26]:
#전처리 끝난 평가 데이터 가져오기
TEST_CLEAN_DATA = 'test_clean.csv'
test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)
test_review = list(test_data['review'])

In [27]:
test_data.head(5)

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


In [28]:
#평가 데이터 역시 학습 데이터 처럼 단어의 리스트로 만들어야 한다
test_sentences = list()
for review in test_review:
  test_sentences.append(review.split())

test_sentences[1]

['movie',
 'disaster',
 'within',
 'disaster',
 'film',
 'full',
 'great',
 'action',
 'scenes',
 'meaningful',
 'throw',
 'away',
 'sense',
 'reality',
 'let',
 'see',
 'word',
 'wise',
 'lava',
 'burns',
 'steam',
 'burns',
 'stand',
 'next',
 'lava',
 'diverting',
 'minor',
 'lava',
 'flow',
 'difficult',
 'let',
 'alone',
 'significant',
 'one',
 'scares',
 'think',
 'might',
 'actually',
 'believe',
 'saw',
 'movie',
 'even',
 'worse',
 'significant',
 'amount',
 'talent',
 'went',
 'making',
 'film',
 'mean',
 'acting',
 'actually',
 'good',
 'effects',
 'average',
 'hard',
 'believe',
 'somebody',
 'read',
 'scripts',
 'allowed',
 'talent',
 'wasted',
 'guess',
 'suggestion',
 'would',
 'movie',
 'start',
 'tv',
 'look',
 'away',
 'like',
 'train',
 'wreck',
 'awful',
 'know',
 'coming',
 'watch',
 'look',
 'away',
 'spend',
 'time',
 'meaningful',
 'content']

In [30]:
#이제 워드투벡을 적용해서 임베딩된 벡터값으로 만들어보자
#이전에 만들어놓은 워드투벡 모델을 적용해서, 위의 각 단어들을 벡터로 만들어 테스트 리뷰의 특징값을 만든다
test_data_vecs = get_dataset(test_sentences, model, num_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [31]:
#이제 이 워드투벡화 된 평가데이터를 가지고 기학습시킨 로지스텍 모델에 적용해보자
#이 평가 데이터는 정답 라벨을 가지고 있지 않기 때문에, 예측한 값을 캐글에 제출한 후 성능 점수를 피드백 받는다
test_predicted = lgs.predict(test_data_vecs)

In [32]:
ids = list(test_data['id'])

answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})

In [33]:
answer_dataset

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1
...,...,...
24995,"""2155_10""",1
24996,"""59_10""",1
24997,"""2531_1""",0
24998,"""7772_8""",1


In [34]:
DATA_OUT_PATH = 'gdrive/My Drive/RogerHeederer/NLP_KR/4.TextClassfication_heederer/data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)